In [47]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from time import time
import pickle
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [3]:
train_data['POI'] = train_data['POI/street'].str.split('/', expand=True)[0]
train_data['street'] = train_data['POI/street'].str.split('/', expand=True)[1]

In [4]:
train_data

,id,raw_address,POI/street,POI,street
0,0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika,,jl kapuk timur delta sili iii lippo cika
1,1,"aye, jati sampurna",/,,
2,2,setu siung 119 rt 5 1 13880 cipayung,/siung,,siung
3,3,"toko dita, kertosono",toko dita/,toko dita,
4,4,jl. orde baru,/jl. orde baru,,jl. orde baru
...,...,...,...,...,...
299995,299995,jend ahmad yani 331 kertasari ciamis,/jend ahmad yani,,jend ahmad yani
299996,299996,"raya cila kko, cilandak timur kel.",/raya cila kko,,raya cila kko
299997,299997,tanjung gusta jl. yaya 2 no 17,/,,
299998,299998,jalan cipadu jaya taman asri gang bijaksana 3 ...,taman asri/,taman asri,


In [5]:
train_data[train_data.apply(lambda x: x['POI'] not in x['raw_address'] ,axis=1)]

,id,raw_address,POI/street,POI,street
10,10,"cikahuripan sd neg boj 02 klap boj, no 5 16877",sd negeri bojong 02/klap boj,sd negeri bojong 02,klap boj
11,11,"yaya atohar,",yayasan atohariyah/,yayasan atohariyah,
20,20,"toko bang ajs,",toko bangunan ajs/,toko bangunan ajs,
40,40,mar tabl metro iringmulyo metro timur,markaz tabligh metro/,markaz tabligh metro,
44,44,sd neg 12 anggrek,sd negeri 12 anggrek/,sd negeri 12 anggrek,
...,...,...,...,...,...
299963,299963,masl letjen sut,maslikah/letjen sut,maslikah,letjen sut
299968,299968,"vok toko, k h mas mans, kampung bali",vokals toko/k h mas mans,vokals toko,k h mas mans
299974,299974,"islamic training cen pare, brawi, pelem pare",islamic training center pare/brawi,islamic training center pare,brawi
299983,299983,"la banda minima, cile raya, pesanggrahan",la banda minimarket/cile raya,la banda minimarket,cile raya


In [19]:
def find_first_index(ra_split, st_split):
    if len(ra_split) >= len(st_split):
        num_iter = len(ra_split) - len(st_split) + 1
        overlap_list = []
        for i in range(num_iter):
            window = ra_split[i: i + len(st_split)]
            overlap = set(window) & set(st_split)
            overlap_list.append(len(overlap))

        max_overlap = [e for e in range(len(overlap_list)) if overlap_list[e] == max(overlap_list)]
        if len(max_overlap)==1:
            return max_overlap[0]
        else:
            count_list = []
            for idx in max_overlap:
                count = 0
                for i in range(len(st_split)):
                    if ra_split[idx+i] in st_split[i]:
                        count+=1
                count_list.append(count)
            index = count_list.index(max(count_list))

            return max_overlap[index]
    else:
        return 0

In [7]:
def fix_street_errors(row):
    raw_add = row['raw_address']
    if row['street'] != '':

        ra_split = word_tokenize(raw_add)
        extr_st = row['street']
        extr_st_split = word_tokenize(extr_st)
        if extr_st in raw_add:
            return raw_add
        else:
            first_index = find_first_index(ra_split, extr_st_split)
            ra_split[first_index: first_index + len(extr_st_split)] = extr_st_split
            update_raw_add = ' '.join(ra_split).replace(' ,', ',').replace(' .', '.').replace(' )', ')').replace(' (', '(').replace(' ?', '?') 
            return update_raw_add
    else:
        return raw_add

In [13]:
start = time()
train_data['cleaned_raw_address'] = train_data.apply(fix_street_errors, axis=1)

print("Executed in {} minutes.".format(round((time() - start)/60, 3)))

# Sanity checks


Executed in 1.271 minutes.


In [14]:
train_data[train_data.apply(lambda x: x['street'] not in x['raw_address'] ,axis=1)]

,id,raw_address,POI/street,POI,street,cleaned_raw_address
69,69,cak 11 nagasari karawang barat,/cakrad,,cakrad,cakrad 11 nagasari karawang barat
86,86,simpang tiga kah nasu no 112 28284 bukit raya,/kaharu nasu,,kaharu nasu,simpang tiga kaharu nasu no 112 28284 bukit raya
117,117,ahmad dah iv kukusan beji,/ahmad dahlan iv,,ahmad dahlan iv,ahmad dahlan iv kukusan beji
130,130,mangla rege no h1 3 cimekar kel. cileunyi,/manglayang rege,,manglayang rege,manglayang rege no h1 3 cimekar kel. cileunyi
135,135,"cipinang besar selatan lintas ibadah, cipi jay...",lintas ibadah/cipinang jaya 1a,lintas ibadah,cipinang jaya 1a,"cipinang besar selatan lintas ibadah, cipinang..."
...,...,...,...,...,...,...
299862,299862,f. l tob 91 batang beruh sidikalang,/f. l tobing,,f. l tobing,f. l tobing 91 batang beruh sidikalang
299870,299870,"wanasari kel. cemp viii, no 12",/cempaka viii,,cempaka viii,"wanasari kel. cempaka viii, no 12"
299910,299910,"kamp lio,",/kampung lio,,kampung lio,"kampung lio,"
299973,299973,moha toha no 167,/mohammad toha,,mohammad toha,mohammad toha no 167


In [17]:
def fix_poi_errors(row):
    raw_add = row['cleaned_raw_address']
    if row['POI'] != '':

        ra_split = word_tokenize(raw_add)
        extr_poi = row['POI']
        extr_poi_split = word_tokenize(extr_poi)
        if extr_poi in raw_add:
            return raw_add
        else:
            first_index = find_first_index(ra_split, extr_poi_split)
            ra_split[first_index: first_index + len(extr_poi_split)] = extr_poi_split
            update_raw_add = ' '.join(ra_split).replace(' ,', ',').replace(' .', '.').replace(' )', ')').replace(' (', '(').replace(' ?', '?') 
            return update_raw_add
    else:
        return raw_add

In [20]:
start = time()
train_data['cleaned_raw_address'] = train_data.apply(fix_poi_errors, axis=1)

print("Executed in {} minutes.".format(round((time() - start)/60, 3)))

# Sanity checks

1
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
5
0
0
0
0
4
0
0
0
0
0
0
0
0
3
0
0
0
0
1
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
1
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
4
0
2
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
6
0
0
5
0
0
0
0
0
0
0
5
0
0
0
0
0
2
0
5
0
4
0
0
0
0
0
10
0
0
0
0
0
0
0
0
6
0
0
0
0
0
0
4
1
0
0
0
4
0
0
4
0
0
0
6
0
0
0
0
0
0
0
6
0
0
4
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
1
6
0
4
0
5
0
6
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
2
0
2
0
0
0
0
0
6
0
2
0
0
0
0
0
4
0
0
0
0
0
0
0
1
0
0
0
0
5
0
0
0
0
0
1
3
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
5
0
3
0
0
0
0
0
2
1
0
0
0
0
4
0
0
0
0
6
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
1
0
0
0
0
0
0
2
0
4
0
0
0
4
0
0
0
2
0
0
0
3
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
6
0
4
0
0
0
1
0
0
0
0
3
0
2
3
6
0
1
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0

0
0
0
0
0
0
0
0
3
1
0
0
3
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
6
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
6
0
0
0
0
0
0
0
5
0
4
0
0
4
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
6
2
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
5
2
0
0
0
0
0
0
5
3
6
0
0
0
0
2
0
0
0
3
0
6
3
0
0
0
0
0
0
2
0
0
0
0
0
6
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
4
0
0
0
0
0
0
0
0
0
0
0
2
0
3
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
2
3
0
0
0
5
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
4
4
3
0
0
0
0
0
0
0
5
0
0
0
2
1
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
6
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
2
0
2
0
0
0
0
0
0
0
0
1
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
4
0
2
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
4
0
0
0
0
5
0
0
2
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
1
0
3
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
4
0
2
0
0


1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
4
0
0
0
0
3
0
0
2
5
6
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
4
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
1
0
0
0
0
0
0
0
7
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
3
0
0
1
0
0
0
0
0
0
0
2
0
0
0
2
5
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
2
0
5
1
6
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
3
6
0
2
0
0
0
0
5
0
0
0
0
2
0
0
0
0
0
0
0
0
0
3
0
0
0
1
3
0
4
0
0
1
0
1
3
0
0
2
4
0
4
0
5
0
3
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
6
0
0
0
0
4
0
0
0
0
0
2
0
3
0
0
0
0
0
2
0
0
0
0
5
0
0
0
7
0
0
0
0
0
0
0
2
0
0
5
0
0
4
0
0
0
0
2
0
2
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
6
0
0
4
2
0
0
1
1
0
0
0
4
0
5
4
0
0
0
0
2
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
6
2
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
5
0
5
0
0
0
0
0
3
0
0
0
0
0
0
5
2
0
0
0
0
0
0
0
0
0
6
0
0
0
2
0
0
0
0
0
0
0
0
0
4
0
0
0
0
2
4
0
0
0
0
0
0
0
0
0
5
0
0
0
4
0
0
0
0
0
0
1
0
0
0
0
0
0
0


0
0
0
0
6
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
3
0
0
0
0
0
0
1
0
3
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
5
0
0
5
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
4
0
0
0
0
0
0
0
0
6
0
1
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
1
0
0
0
0
1
0
2
0
0
0
0
0
4
0
1
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
4
5
3
0
0
0
0
0
0
0
0
7
6
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
6
0
2
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
5
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
8
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
7
0
0
4
0
0
1
4
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
3
0
1
0
0
0
0
0
0
5
0
0
0
0
0
6
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
7
0
0
0
0
0
0
0
0
5
0
2
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
2
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
1
0
0
0


0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
3
0
0
0
0
0
0
0
0
5
0
0
0
4
0
0
0
0
0
0
3
0
5
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
4
0
2
0
3
0
0
0
0
4
7
0
0
1
0
0
0
0
5
3
0
0
2
0
5
0
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
6
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
6
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
4
0
0
0
0
0
4
0
0
0
0
0
2
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
2
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
7
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
1
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
7
0
0
1
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
8
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
2
0
0
3
0
0
0
0
0
0
5
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
4
0
0
2
0
0
0
0
0
0
3
0
0
0
0
0
0
9
0
4
0
0
0
0
0
0
5
3
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
2
0
0
0
0
0


...

6
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
2
0
0
0
5
0
0
0
4
7
0
2
0
1
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
4
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
4
0
0
0
0
5
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
5
0
0
3
4
0
0
2
0
0
0
0
0
0
7
0
0
0
0
6
0
0
0
0
0
0
0
0
0
0
8
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
3
0
0
12
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
2
0
1
0
5
0
0
2
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
3
0
0
1
0
6
0
6
0
0
0
0
0
0
3
3
0
0
4
0
0
0
0
0
0
0
0
0
2
0
0
3
4
0
0
0
3
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
5
4
3
4
0
0
0
0
1
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
3
0
0
0
0
0
0
0
2
0
0
2
0
0
0
0
0
0
0
7
0
0
0
3
0
0
6
0
0
0
0
3
0
4
0
0
0
0
0
0
5
0
0
0
0
0
0
0
5
0
0
0
0
2
0
0
0
0
0
0
4
0
0
0
0
0
0
0
5
5
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
5
6
0
0
1
0
0
3
0
0
7
0
0
0
0
0
0
0
0
0
1
1
0
1
0
0
0
0
0
0
0
7
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0

0
5
0
0
4
0
0
0
0
0
4
1
0
0
0
3
5
0
0
1
0
0
4
4
1
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
6
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
6
4
0
5
0
0
0
0
0
0
0
0
6
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
3
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
5
5
0
0
0
0
0
0
0
0
0
0
4
4
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
3
0
4
0
0
0
0
0
0
0
0
0
0
0
1
3
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
5
0
0
0
0
2
0
6
0
0
4
0
0
0
3
0
0
3
0
0
0
6
5
0
0
5
0
0
0
0
0
0
0
3
0
0
0
0
0
0
1
0
0
0
0
0
0
0
3
0
0
0
0
0
5
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
3
0
0
4
0
0
0
4
3
0
0
0
0
0
0
0
0
0
1
1
4
0
0
2
0
0
0
0
0
0
0
0
0
6
0
1
0
0
4
0
0
4
0
0
0
5
0
0
0
0
0
0
3
0
0
0
0
3
0
0
0
0
0
0
1
4
0
1
2
0
0
0
0
1
0
0
0
0
0
0
0
0
2
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
3
0
1
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
1
0
0
0
3
0
0


0
0
0
0
1
0
0
0
0
0
2
0
0
0
0
0
2
3
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
5
0
0
5
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
2
0
3
0
0
0
0
0
0
7
0
0
7
4
0
0
0
0
0
0
0
1
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
6
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
3
0
0
0
2
0
0
0
0
0
0
0
0
0
0
4
5
0
0
4
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
1
0
0
0
0
0
0
0
0
0
7
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
6
0
4
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
4
0
0
0
0
0
1
0
5
0
0
0
0
2
0
0
0
0
0
0
0
5
4
0
0
0
0
0
0
0
0
0
3
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
1
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
2
0
0
4
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
1
0
0
0
0
0
0
0
0
5
0
0
1
0
0
0
0
3
0
0
0
5
0
0
0
3
0
0
0
0
9
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
6
10
0
0
4
0
0
0
0
0
6
0
0
5

0
5
0
6
0
0
0
0
0
1
0
0
4
0
0
0
0
0
0
3
0
0
1
0
0
0
1
4
0
0
0
0
2
0
0
5
2
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
2
5
3
0
0
4
0
0
0
7
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
5
0
5
0
0
0
0
0
0
0
0
7
0
0
5
0
0
0
0
5
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
3
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
6
0
0
0
0
0
4
0
0
0
0
0
0
2
0
0
0
2
0
0
3
0
2
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
3
0
0
0
5
0
0
0
1
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
7
3
0
0
0
0
0
0
0
0
0
0
3
0
0
0
4
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
1
0
0
0
4
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
3
0
0
0
0
0
5
0
0
7
0
7
0
0
7
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
8
0
0
0
0
0
0
0
6
0
0
0
0
0
4
0
0
5
0
0
0
0
0
6
0
0
0
0
0
0
0
0
0
0
0
0
0
5
3
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0


0
0
0
2
0
1
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
3
0
0
0
2
0
7
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
1
0
3
0
0
0
0
6
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
6
0
0
0
0
1
0
0
0
0
6
5
0
5
4
0
0
3
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
8
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
1
0
0
0
0
0
4
0
0
4
0
0
0
4
0
0
0
0
0
0
0
0
0
4
4
0
0
0
3
7
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
6
0
7
0
0
0
0
0
0
0
0
0
6
0
2
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
6
0
0
0
0
0
0
2
0
1
0
0
0
4
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
2
0
0
1
0
0
0
0
5
6
0
0
0
2
0
0
3
0
0
0
0
1
0
0
0
6
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
3
2
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
2
5
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
4
0
0
5
0
0
6
0
0
0
0
0
0
0
0
0
2
0
0
0
0
5
0
5
0
4
0
0
5
0
0
0
0
0
0
0
0
0
5
0
0
4
0
0
3
0
0
0
1
3
0
0
0
0
1
5
0
0
0
0
0
3
0
0
5
0
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
6
0
2
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
5
0
0
5
0
0
0
0
0


In [21]:
train_data[train_data.apply(lambda x: x['POI'] not in x['raw_address'] ,axis=1)]

,id,raw_address,POI/street,POI,street,cleaned_raw_address
10,10,"cikahuripan sd neg boj 02 klap boj, no 5 16877",sd negeri bojong 02/klap boj,sd negeri bojong 02,klap boj,"cikahuripan sd negeri bojong 02 klap boj, no 5..."
11,11,"yaya atohar,",yayasan atohariyah/,yayasan atohariyah,,"yayasan atohariyah,"
20,20,"toko bang ajs,",toko bangunan ajs/,toko bangunan ajs,,"toko bangunan ajs,"
40,40,mar tabl metro iringmulyo metro timur,markaz tabligh metro/,markaz tabligh metro,,markaz tabligh metro iringmulyo metro timur
44,44,sd neg 12 anggrek,sd negeri 12 anggrek/,sd negeri 12 anggrek,,sd negeri 12 anggrek
...,...,...,...,...,...,...
299963,299963,masl letjen sut,maslikah/letjen sut,maslikah,letjen sut,maslikah letjen sut
299968,299968,"vok toko, k h mas mans, kampung bali",vokals toko/k h mas mans,vokals toko,k h mas mans,"vokals toko, k h mas mans, kampung bali"
299974,299974,"islamic training cen pare, brawi, pelem pare",islamic training center pare/brawi,islamic training center pare,brawi,"islamic training center pare, brawi, pelem pare"
299983,299983,"la banda minima, cile raya, pesanggrahan",la banda minimarket/cile raya,la banda minimarket,cile raya,"la banda minimarket, cile raya, pesanggrahan"


In [27]:
def get_street_mapping_dict(row):
    raw_add = row['raw_address']
    if row['street'] != "":
        ra_split = word_tokenize(raw_add)
        extr_st = row['street']
        extr_st_split = word_tokenize(extr_st)
        if extr_st in raw_add:
            return None
        else:
            first_index = find_first_index(ra_split, extr_st_split)
            before = ra_split[first_index: first_index + len(extr_st_split)]            
            before = ' '.join(before)
            return before, extr_st
    else:
        return None

In [28]:
start = time()
train_data['street_mapping'] = train_data.apply(get_street_mapping_dict, axis=1)

print("Executed in {} minutes.".format(round((time() - start)/60, 3)))

# Sanity checks

Executed in 1.061 minutes.


In [30]:
train_data[train_data.apply(lambda x: x['street'] not in x['raw_address'] ,axis=1)]

,id,raw_address,POI/street,POI,street,cleaned_raw_address,street_mapping
69,69,cak 11 nagasari karawang barat,/cakrad,,cakrad,cakrad 11 nagasari karawang barat,"(cak, cakrad)"
86,86,simpang tiga kah nasu no 112 28284 bukit raya,/kaharu nasu,,kaharu nasu,simpang tiga kaharu nasu no 112 28284 bukit raya,"(kah nasu, kaharu nasu)"
117,117,ahmad dah iv kukusan beji,/ahmad dahlan iv,,ahmad dahlan iv,ahmad dahlan iv kukusan beji,"(ahmad dah iv, ahmad dahlan iv)"
130,130,mangla rege no h1 3 cimekar kel. cileunyi,/manglayang rege,,manglayang rege,manglayang rege no h1 3 cimekar kel. cileunyi,"(mangla rege, manglayang rege)"
135,135,"cipinang besar selatan lintas ibadah, cipi jay...",lintas ibadah/cipinang jaya 1a,lintas ibadah,cipinang jaya 1a,"cipinang besar selatan lintas ibadah, cipinang...","(cipi jaya 1a, cipinang jaya 1a)"
...,...,...,...,...,...,...,...
299862,299862,f. l tob 91 batang beruh sidikalang,/f. l tobing,,f. l tobing,f. l tobing 91 batang beruh sidikalang,"(f. l tob, f. l tobing)"
299870,299870,"wanasari kel. cemp viii, no 12",/cempaka viii,,cempaka viii,"wanasari kel. cempaka viii, no 12","(cemp viii, cempaka viii)"
299910,299910,"kamp lio,",/kampung lio,,kampung lio,"kampung lio,","(kamp lio, kampung lio)"
299973,299973,moha toha no 167,/mohammad toha,,mohammad toha,mohammad toha no 167,"(moha toha, mohammad toha)"


In [31]:
def get_poi_mapping_dict(row):
    raw_add = row['raw_address']
    if row['POI'] != "":
        ra_split = word_tokenize(raw_add)
        extr_poi = row['POI']
        extr_poi_split = word_tokenize(extr_poi)
        if extr_poi in raw_add:
            return None
        else:
            first_index = find_first_index(ra_split, extr_poi_split)
            before = ra_split[first_index: first_index + len(extr_poi_split)]            
            before = ' '.join(before)
            return before, extr_poi
    else:
        return None

In [32]:
start = time()
train_data['poi_mapping'] = train_data.apply(get_poi_mapping_dict, axis=1)

print("Executed in {} minutes.".format(round((time() - start)/60, 3)))

# Sanity checks

Executed in 0.634 minutes.


In [33]:
train_data[train_data.apply(lambda x: x['POI'] not in x['raw_address'] ,axis=1)]

,id,raw_address,POI/street,POI,street,cleaned_raw_address,street_mapping,poi_mapping
10,10,"cikahuripan sd neg boj 02 klap boj, no 5 16877",sd negeri bojong 02/klap boj,sd negeri bojong 02,klap boj,"cikahuripan sd negeri bojong 02 klap boj, no 5...",None,"(sd neg boj 02, sd negeri bojong 02)"
11,11,"yaya atohar,",yayasan atohariyah/,yayasan atohariyah,,"yayasan atohariyah,",None,"(yaya atohar, yayasan atohariyah)"
20,20,"toko bang ajs,",toko bangunan ajs/,toko bangunan ajs,,"toko bangunan ajs,",None,"(toko bang ajs, toko bangunan ajs)"
40,40,mar tabl metro iringmulyo metro timur,markaz tabligh metro/,markaz tabligh metro,,markaz tabligh metro iringmulyo metro timur,None,"(mar tabl metro, markaz tabligh metro)"
44,44,sd neg 12 anggrek,sd negeri 12 anggrek/,sd negeri 12 anggrek,,sd negeri 12 anggrek,None,"(sd neg 12 anggrek, sd negeri 12 anggrek)"
...,...,...,...,...,...,...,...,...
299963,299963,masl letjen sut,maslikah/letjen sut,maslikah,letjen sut,maslikah letjen sut,None,"(masl, maslikah)"
299968,299968,"vok toko, k h mas mans, kampung bali",vokals toko/k h mas mans,vokals toko,k h mas mans,"vokals toko, k h mas mans, kampung bali",None,"(vok toko, vokals toko)"
299974,299974,"islamic training cen pare, brawi, pelem pare",islamic training center pare/brawi,islamic training center pare,brawi,"islamic training center pare, brawi, pelem pare",None,"(islamic training cen pare, islamic training c..."
299983,299983,"la banda minima, cile raya, pesanggrahan",la banda minimarket/cile raya,la banda minimarket,cile raya,"la banda minimarket, cile raya, pesanggrahan",None,"(la banda minima, la banda minimarket)"


In [34]:
df_street_mapping = train_data[train_data['street_mapping'].notnull()]
street_mapping_dict=dict()
for row in df_street_mapping['street_mapping']:
    street_mapping_dict[row[0]] = row[1]


In [35]:
street_mapping_dict

{'cak': 'cakrab',
 'kah nasu': 'kaharu nasu',
 'ahmad dah iv': 'ahmad dahlan iv',
 'mangla rege': 'mangla regency',
 'cipi jaya 1a': 'cipinang jaya 1a',
 'taman mer': 'taman meruya',
 'man kerto v': 'manyar kerto v',
 'par': 'pariwi',
 'pahl': 'pahlawan',
 'yos suda': 'yos sudarso',
 'putri gad cemp': 'putri gad cempaka',
 'parangt': 'parangtritis',
 'hus sast': 'husein sastran',
 'pulori': 'puloriburit',
 'kar utara': 'karang utara',
 'komp bumi panyileukan': 'komplek bumi panyileukan',
 'jend gatot subr': 'jendral gatot subr',
 'raya kalija': 'raya kalijati-p',
 'r. wol mongin': 'r. wol monginsidi',
 'waturen': 'waturenggong',
 'dok wahi': 'dokter wahidin',
 'raya banjarn - pur': 'raya banjarnegara - purwok',
 'ir suci': 'ir sucitro',
 'macc sawah': 'maccini sawah',
 'raya ubu': 'raya ubung-',
 'colu ten ii': 'columbus ten ii',
 'bule timur kel gading': 'bulevar timur kel gading',
 'kolo sugi': 'kolo sugiono',
 'macc gus': 'macc gusung',
 'lin timur': 'lintas timur',
 'pala tent pela

In [36]:
df_poi_mapping = train_data[train_data['poi_mapping'].notnull()]
poi_mapping_dict=dict()
for row in df_poi_mapping['poi_mapping']:
    poi_mapping_dict[row[0]] = row[1]


In [37]:
poi_mapping_dict

{'sd neg boj 02': 'sd negeri bojong 02',
 'yaya atohar': 'yayasan atohariyah',
 'toko bang ajs': 'toko bangunan ajs',
 'mar tabl metro': 'markaz tabligh metro',
 'sd neg 12 anggrek': 'sd negeri 12 anggrek',
 'rumah makan pela': 'rumah makan pelangi',
 'rnd prin': 'rnd printing',
 'pp minhajutt': 'pp minhajutthollab',
 'tk islam daruss': 'tk islam darussalam',
 'bint jaya ud': 'bintang jaya ud',
 'cau terr': 'causal terrace',
 'kedai tenun jep senn': 'kedai tenun jepara sennaart',
 'cv . hin oto kenc': 'cv. hingdi oto kencana',
 'caval pizz': 'cavallino pizzaria',
 'grey': 'grey101',
 'beng las deny': 'bengkel las deny',
 'ber nadah agung': 'berkat nadah agung',
 'sd bust islamiah': 'sd bustanul islamiah',
 'sd neg palumbungan 1': 'sd negeri palumbungan 1',
 'pd . karya warga mand cimb': 'pd. karya warga mandiri cimb',
 'prim warna dan rekan': 'primaadi warna dan rekan',
 'alun - alun kuli nusan': 'alun - alun kuliner nusantara',
 'kantor bupati tapa sela': 'kantor bupati tapanuli selat

In [45]:
clean_df = train_data[['id', 'cleaned_raw_address', 'POI', 'street']]
clean_df.columns = ['id', 'raw_address', 'POI', 'street']
clean_df

,id,raw_address,POI,street
0,0,jl kapuk timur delta sili iii lippo cika 11 a ...,,jl kapuk timur delta sili iii lippo cika
1,1,"aye, jati sampurna",,
2,2,setu siung 119 rt 5 1 13880 cipayung,,siung
3,3,"toko dita, kertosono",toko dita,
4,4,jl. orde baru,,jl. orde baru
...,...,...,...,...
299995,299995,jend ahmad yani 331 kertasari ciamis,,jend ahmad yani
299996,299996,"raya cila kko, cilandak timur kel.",,raya cila kko
299997,299997,tanjung gusta jl. yaya 2 no 17,,
299998,299998,jalan cipadu jaya taman asri gang bijaksana 3 ...,taman asri,


In [46]:
clean_df.to_csv('clean_train.csv', index=False)

In [48]:
# Save the dictionaries
s_file = open("street_mapping_dict.pkl", "wb")
pickle.dump(street_mapping_dict, s_file)
s_file.close()

p_file = open("poi_mapping_dict.pkl", "wb")
pickle.dump(street_mapping_dict, p_file)
p_file.close()

In [43]:
test_df = pd.read_csv('test.csv')
test_df

,id,raw_address
0,0,s. par 53 sidanegara 4 cilacap tengah
1,1,"angg per, baloi indah kel. lubuk baja"
2,2,"asma laun, mand imog,"
3,3,"ud agung rej, raya nga sri wedari karanganyar"
4,4,"cut mutia, 35 baiturrahman"
...,...,...
49995,49995,toko mbak farid semboro semboro
49996,49996,"vie - tk. ridho kids, vete 3 cari, 16720 ciawi"
49997,49997,"mart dan roti bakar malabar, nasio,"
49998,49998,graha indah pamulang jl. mujair raya bambu apu...


In [40]:
new_street_mapping_dict = {k: v for k, v in street_mapping_dict.items() if len(k.split())>1}
new_poi_mapping_dict = {k: v for k, v in poi_mapping_dict.items() if len(k.split())>1}


In [41]:
len(new_street_mapping_dict)

9111

In [49]:
start = time()
# Replace truncated words in raw_address of test set with correct street labels
count1 = 0
for row, col in test_df.iterrows():
    for k, v in new_street_mapping_dict.items():
        if k in col['raw_address']:
            test_df.loc[row, 'raw_address'] = test_df.loc[row, 'raw_address'].replace(k, v)
            count1 += 1
            
print("Number of raw addresses updated due to errors in street labels:", count1)

# Replace truncated words in raw_address of test set with correct POI labels
count2 = 0
for row, col in test_df.iterrows():
    for k, v in new_poi_mapping_dict.items():
        if k in col['raw_address']:
            test_df.loc[row, 'raw_address'] = test_df.loc[row, 'raw_address'].replace(k, v)
            count2 += 1

print("Number of raw addresses updated due to errors in POI labels:", count2)

print("Executed in {} minutes.".format(round((time() - start)/60, 3)))

test_df

Number of raw addresses updated due to errors in street labels: 18913
Number of raw addresses updated due to errors in POI labels: 4544
Executed in 507.504 minutes.


,id,raw_address
0,0,s. parman 53 sidanegara 4 cilacap tengah
1,1,"anggrek per, baloi indah kel. lubuk baja"
2,2,"asma laun, mangund imog,"
3,3,"ud agung rejeki, raya ngawi- sri wedari karang..."
4,4,"cut mutia, 35 baiturrahman"
...,...,...
49995,49995,toko mbak farid semboro semboro
49996,49996,"vie - tk. ridho kids, vete 3 cari, 16720 ciawi"
49997,49997,"mart dan roti bakar malabar, nasio,"
49998,49998,graha indah pamulang jl. mujair raya bambanbu ...


In [50]:
test_df.to_csv('clean_test.csv', index=False)